# How to install Jupyter on Virtualbox with Vagrant

We are going to install Jupyter Lab on to a Virtualbox VM with Vagrant and Ansible.

**WHY?**
- Make it easy for someone to test out HashiCorp Solutions with Jupyter Notebooks.
    - currently, you would need to install lots of prerequisites to get going with JupyterLab and Hashi products.
    - the VM provisioned with Vagrant will come pre-loaded with these pre-requisites:
        - consul, nomad, vault
        - docker
        - (soon) cloud cli's - aws, azure, etc

**HOW?**
1. We will bring up an Ubuntu 20 VM with Vagrant.
1. We run an Ansible playbook to install:
    - Basics: jq, tree, curl, etc
    - Hashi: consul, nomad, vault
    - JupyterLab
    - Clone hc_demos-jupyter and hashistack-ansible
    - (soon) cloud cli's, ansible, and more
1. Vagrant will publish port 8888 on the VM.
1. You connect to the VM's private address on that port to get to JupyterLab
1. JupyterLab will come up with the notebook repo mounted.

## Pre-requisites

- Software:
    - Virtualbox
    - Vagrant
    - Ansible
- These GitHub repos:
    - `phanclan/hc_demos-jupyter`
    - `phanclan/hashistack-ansible`

### General Setup

In [1]:
SCRIPTS_DIR=./Scripts

In [ ]:
#==> Seet baseline script.
cat ${SCRIPTS_DIR}/baseline.sh

In [2]:
#==> Set shell customizations
source ${SCRIPTS_DIR}/baseline.sh


#==> Set colors.

#==> Some commands may have sensitive information.
#==> Prevent commands starting with a space to be saved to shell history.


### Install Virtualbox

Install VirtualBox if you don't have it.
- Download: https://www.virtualbox.org/wiki/Downloads
- Installation Instructions: https://www.virtualbox.org/manual/UserManual.html#installation

Here are a few methods to install on Linux.

#### Install VirtualBox on Ubuntu 20.04

This method is easiest for Linux but you get an older version of VirtualBox.

In [12]:
DEBIAN_FRONTEND=noninteractive
sudo apt-get -qq update > /dev/null
sudo apt-get -qq install virtualbox
# can't install virtualbox-ext-pack via Jupyter.
# Need to run in separate terminal.
# echo sudo apt-get -qq install virtualbox-ext-pack || true

The output shows that you have successfully installed "Oracle VM VirtualBox Extension Pack".

#### Install VirtualBox from virtualbox.org on Ubuntu

This method adds Oracle's repository. You get the latest version and can update with system updates.

In [ ]:
#==> 1. Import and add the GPG repository keys to your system.
wget -q https://www.virtualbox.org/download/oracle_vbox_2016.asc -O- | sudo apt-key add -

In [ ]:
#==> 2. Add the VirtualBox APT repository.
sudo add-apt-repository "deb [arch=amd64] http://download.virtualbox.org/virtualbox/debian $(lsb_release -cs) contrib"

In [ ]:
#==> 3. Update the APT packages cache repository.
sudo apt-get update

In [ ]:
#==> 4. Install the latest version of VirtualBox.
sudo apt-get -qq install virtualbox-6.1

In [ ]:
#==> 5. (optional) Install VirtualBox Extension Pack
# curl -o /tmp/Oracle_VM_VirtualBox_Extension_Pack-6.1.30.vbox-extpack \
#     -L https://download.virtualbox.org/virtualbox/6.1.30/Oracle_VM_VirtualBox_Extension_Pack-6.1.30.vbox-extpack

# yes | sudo VBoxManage extpack install \
#     --accept-license=33d7284dc4a0ece381196fda3cfe2ed0e1e8e7ed7f27b9a9ebc4ee22e24bd23c \
#     /tmp/Oracle_VM_VirtualBox_Extension_Pack-6.1.30.vbox-extpack

The above command works with sudo but the extensions only show up for sudo.

In [ ]:
VBoxManage list extpacks

In [ ]:
sudo VBoxManage list extpacks

### Install Vagrant

Install Vagrant if you don't have it.

- Download and Instructions: https://www.vagrantup.com/downloads

Here is an install method for Ubuntu.

#### Install Vagrant on Ubuntu

The instructions below are for Ubuntu. For other distributions, go [here](https://www.vagrantup.com/downloads).

In [ ]:
curl -fsSL https://apt.releases.hashicorp.com/gpg | sudo apt-key add -
sudo apt-add-repository "deb [arch=amd64] https://apt.releases.hashicorp.com $(lsb_release -cs) main"
sudo apt-get update && sudo apt-get install vagrant

### Install Ansible

We will use Ansible to provision the VM's that Vagrant creates. You will need `python` and `pip` for this.

In [ ]:
# mkdir -p ansible
pip install ansible

In [ ]:
printf "${GREEN_REVERSE}#==> Confirm Install${NC}\n"
ansible --version

# Ansible

In [ ]:
SCRIPTS_DIR=/media/code/hc_demos-jupyter/Scripts

## Clone Ansible Repo from phanclan

The `phanclan/hashistack-ansible` has a few handy Ansible roles that will configure your VM for you.

1. Clone `planclan/hashistack-ansible`.
1. Be sure to change the repo directory `hashistack_ansible_repo` to your own path.
1. Make sure it is at the same directory level as `hc_demos-jupyter` directory.

In [119]:
hashistack_ansible_repo=/media/code/hashistack-ansible

In [ ]:
mkdir -p ${hashistack_ansible_repo}
git clone https://github.com/phanclan/hashistack-ansible ${hashistack_ansible_repo} || true

## Create Ansible Playbook

This playbook will run several roles and references a few variables files. We will create the variables files afterwards.

Make sure the ansible directory is in the expected location.

In [116]:
pwd
ls -l ../hashistack-ansible/ansible/

/media/code/hc_demos-jupyter
total 100
-rwx------ 1 pephan pephan  333 Mar 20  2021 ansible.cfg
-rwx------ 1 pephan pephan  754 Dec 14 16:27 Dockerfile
-rwx------ 1 pephan pephan 2606 Apr 12  2021 hosts
-rwx------ 1 pephan pephan  519 Mar 21  2021 hosts.instruqt
-rwx------ 1 pephan pephan 2951 May 12  2021 hosts_multipass
-rwx------ 1 pephan pephan 1205 May 12  2021 play-hashistack.yml
-rwx------ 1 pephan pephan 1242 Feb 15  2021 play-hashi-test.yml
-rwx------ 1 pephan pephan  359 Feb 15  2021 play-hashivault.yml
-rwx------ 1 pephan pephan  528 Mar 20  2021 play-hashi.yml
-rw-rw-r-- 1 pephan pephan 1318 May 27 16:11 play_install_jupyter.yml
-rwx------ 1 pephan pephan  443 May 26 22:52 play_jupyter.yml
-rwx------ 1 pephan pephan  118 Feb 15  2021 play-multipass.yml
-rwx------ 1 pephan pephan  347 Feb 15  2021 play-nomad-instruqt.yml
-rwx------ 1 pephan pephan  335 Mar 18 08:11 play_playground.yml
-rwx------ 1 pephan pephan 5773 Dec 14 16:26 play-test.yml
-rwx------ 1 pephan pephan  339 

Create ansible playbook `play_install_jupyter.yml`.

In [154]:
cat > ${hashistack_ansible_repo}/ansible/play_install_jupyter.yml <<"EOF"
---
- hosts: dc1-s-1 #Specify host/group.
  become: true
  vars_files:
    - /tmp/vars_setup_ubuntu.yml
    #- vars/vars_setup_consul.yml
  roles:
    - base
    - docker
    # - consul
    # - nomad
  tasks:
    - name: Git checkout hc_demos-jupyter
      become: false
      ansible.builtin.git:
        repo: 'https://github.com/phanclan/hc_demos-jupyter.git'
        dest: /home/vagrant/hc_demos-jupyter
        single_branch: yes
    - name: Git checkout hashistack-ansible
      become: false
      ansible.builtin.git:
        repo: 'https://github.com/phanclan/hashistack-ansible.git'
        dest: /home/vagrant/hashistack-ansible
        single_branch: yes
    - name: Install JupyterLab
      become: false
      shell: |
        pip install --upgrade jinja2 >> /tmp/ansible.out 2>&1
        pip install jupyterlab >> /tmp/ansible.out 2>&1
        pip install bash_kernel >> /tmp/ansible.out 2>&1
        python3 -m bash_kernel.install >> /tmp/ansible.out 2>&1
    - name: Start JupyterLab
      become: false
      shell: |
        export PATH="$HOME/.local/bin:$PATH"
        nohup /home/vagrant/.local/bin/jupyter lab --ip=0.0.0.0 --port=8888 --no-browser --notebook-dir=/home/vagrant >> /tmp/jupyterlab.out 2>&1 &
EOF

Review the file

In [155]:
cat ${hashistack_ansible_repo}/ansible/play_install_jupyter.yml

---
- hosts: dc1-s-1 #Specify host/group.
  become: true
  vars_files:
    - /tmp/vars_setup_ubuntu.yml
    #- vars/vars_setup_consul.yml
  roles:
    - base
    - docker
    # - consul
    # - nomad
  tasks:
    - name: Git checkout hc_demos-jupyter
      become: false
      ansible.builtin.git:
        repo: 'https://github.com/phanclan/hc_demos-jupyter.git'
        dest: /home/vagrant/hc_demos-jupyter
        single_branch: yes
    - name: Git checkout hashistack-ansible
      become: false
      ansible.builtin.git:
        repo: 'https://github.com/phanclan/hashistack-ansible.git'
        dest: /home/vagrant/hashistack-ansible
        single_branch: yes
    - name: Install JupyterLab
      become: false
      shell: |
        pip install --upgrade jinja2 >> /tmp/ansible.out 2>&1
        pip install jupyterlab >> /tmp/ansible.out 2>&1
        pip install bash_kernel >> /tmp/ansible.out 2>&1
        python3 -m bash_kernel.install >> /tmp/ansible.out 2>&1
    - name: Start JupyterLab

What is this playbook doing?
* `hosts: all` apply tasks to `all` hosts in inventory file.
* `vars/vars_setup_ubuntu.yml` - we will set generic variables here.
* `vars/vars_setup_consul.yml` - we will set hashi specific variables here.
* `roles` - specify the roles we want to apply.
    * I comment some roles out when i want to test or reapply a subset.
    * `base` - installs prerequisites and hashistack (consul, nomad, vault)
    * `docker` - installs docker and docker-compose
    * `consul` - configures consul basics
    * `nomad` - configures nomad basics

## Create Ansible Variable Files

These variables are for the `base` and `docker` roles.

In [ ]:
consul_license_path=${HOME}/Downloads/consul_prem_20230111.hclic
nomad_license_path=${HOME}/Downloads/nomad_prem_20230111.hclic

### Base Variables vars_setup_ubuntu.yml

You can accept most defaults, but you must make sure the following is correct:
* `os_interface`
* `arch` - in case you are using **ARM** cpu
* `user` - default os user

In [47]:
DOCKER_REGISTRY=192.168.17.170:5000
# mkdir -p ../../../hashistack-ansible/ansible/vars
cat > /tmp/vars_setup_ubuntu.yml <<EOF
arch: amd64 # arm64, amd64
create_user: pephan
os_interface: enp0s8 #enp0s2, enp0s8:vagrant ubuntu public
user: vagrant

region_letter: a
server_prefix: dc1-s-
client_prefix: dc1-s- # i use the same prefix as server since my nodes do dual roles.

# See vars-instruqt.yml for common variables.
base_apps_list:
  - jq
  - git
  - tree
  - unzip
  - wget
  - curl
  - net-tools
  - python3-pip
apps_hashi:
  - consul-enterprise=1.9.1+ent
  - nomad-enterprise=1.2.3+ent #1.0.15+ent
  - vault-enterprise=1.7.8+ent
copy_local_key: "{{ lookup('file', lookup('env','HOME') + '/.ssh/id_rsa.pub') }}"
docker_registry: ${DOCKER_REGISTRY}

create_containers: 3
default_container_name: docker
default_container_image: hashicorp/consul
default_container_command: consul agent -dev

host_file:
  dc1-s-1:
    ip: "192.168.17.111"
  dc1-s-2:
    ip: "192.168.17.112"
  dc1-s-3:
    ip: "192.168.17.113"
  # server-b-1:
  #   ip: 192.168.64.28
  # server-b-2:
  #   ip: 192.168.64.27
  # server-b-3:
  #   ip: 192.168.64.29
  # client-a-1:
  #   ip: 192.168.64.17
  # client-a-2:
  #   ip: 192.168.64.18
  # client-a-3:
  #   ip: 192.168.64.35
EOF

### Hashi Variables vars_setup_consul.yml

In [ ]:
export CONSUL_DC_1=zone1 CONSUL_DC_2=zone2 CONSUL_DC_3=zone3

In [19]:
# mkdir -p ../../../hashistack-ansible/ansible/vars
tee /tmp/vars_setup_consul.yml <<EOF
consul_datacenter: ${CONSUL_DC_1}
consul_license_path: ${consul_license_path}
nomad_license_path: ${nomad_license_path}
nomad_datacenter: ${CONSUL_DC_1}
consul_retry_join: ' ["192.168.17.111","192.168.17.112","192.168.17.113"]'
EOF

mkdir: cannot create directory ‘../../../hashistack-ansible’: Permission denied
consul_datacenter: 
consul_license_path: 
nomad_license_path: 
nomad_datacenter: 
consul_retry_join: ' ["192.168.17.111","192.168.17.112","192.168.17.113"]'


# Vagrant

Go to `Vagrant/tmp` directory.

In [ ]:
cd ../Vagrant/tmp

In [ ]:
pwd; tree

Expected Output
```shell
/media/code/hc_demos-jupyter/Vagrant/tmp
.
├── ansible
│   ├── Pipfile
│   └── Pipfile.lock
├── ansible.cfg
├── html
│   └── index.html
├── Pipfile
├── playbook.yml
├── Vagrantfile
└── Vagrantfile.example

2 directories, 8 files
```

## Create Vagrantfile

Open `Vagrantfile` if already created.

In [ ]:
cat Vagrantfile

In [22]:
echo $SCRIPTS_DIR

./Scripts


Need to make bridge interface a variable. `enp0s3` for ubuntu 20 vb vm.

In [30]:
OS_INTERFACE=enp0s3
mkdir -p tmp

In [59]:
cat ../hashistack-ansible/ansible/play_install_jupyter.yml

---
- hosts: localhost #all #Specify host/group.
  become: true
  vars_files:
    - /tmp/vars_setup_ubuntu.yml
    #- vars/vars_setup_consul.yml
  roles:
    - base
    - docker
    #- consul
    #- nomad
  tasks:
    - name: test shell
      shell: |
        hostname
  #   - name: Create docker daemon file
  #     template:
  #       src: daemon.json.j2
  #       dest: /etc/docker/daemon.json
  #   - name: Enable and start docker service
  #     service:
  #       name: docker
  #       state: restarted


In [87]:
tee tmp/Vagrantfile <<EOF
SCRIPTS_DIR="${SCRIPTS_DIR}"
Vagrant.configure("2") do |config|
  #==> Configure VM resources
  config.vm.box = "ubuntu/focal64"
  config.vm.provider :virtualbox do |vb|
    vb.cpus = 2
    vb.memory = 4096
  end

  N=3
  # How many nodes do we want?
  #==> 3-node - DC1
  (1..1).each do |node|
    config.vm.define "dc1-s-#{node}" do |s|
      s.vm.host_name = "dc1-s-#{node}"
      
      s.vm.network "private_network", type: "dhcp"
      s.vm.network "forwarded_port", guest: 8889, host: 8888, auto_correct: true
      
      # s.vm.network "public_network", bridge: "${OS_INTERFACE}",
      #   use_dhcp_assigned_default_route: true,
      #   ip: "192.168.17.#{node+110}"
      
      # if node == 1
      #   # Expose the nomad ports
      #   n.vm.network "forwarded_port", guest: 4646, host: 4646, auto_correct: true
      # end
      #==> Ansible Start
      if node == 1
        s.vm.provision "ansible" do |ansible|
          # Disable default limit to connect to all the machines
          ansible.limit = "all"
          # ansible.playbook = "playbook.yml"
          ansible.playbook = "../../hashistack-ansible/ansible/play_install_jupyter.yml"
          ansible.groups = {
            "dc1_clients" => ["dc1-c-[1:2]"],
            "all:vars" => {"arch" => "amd64",
            }
          }
        end
      end
    end
  end
end
EOF

SCRIPTS_DIR="./Scripts"
Vagrant.configure("2") do |config|
  #==> Configure VM resources
  config.vm.box = "ubuntu/focal64"
  config.vm.provider :virtualbox do |vb|
    vb.cpus = 2
    vb.memory = 4096
  end

  N=3
  # How many nodes do we want?
  #==> 3-node - DC1
  (1..1).each do |node|
    config.vm.define "dc1-s-#{node}" do |s|
      s.vm.host_name = "dc1-s-#{node}"
      
      s.vm.network "private_network", type: "dhcp"
      s.vm.network "forwarded_port", guest: 8888, host: 8888, auto_correct: true
      
      # s.vm.network "public_network", bridge: "enp0s3",
      #   use_dhcp_assigned_default_route: true,
      #   ip: "192.168.17.#{node+110}"
      
      # if node == 1
      #   # Expose the nomad ports
      #   n.vm.network "forwarded_port", guest: 4646, host: 4646, auto_correct: true
      # end
      #==> Ansible Start
      if node == 1
        s.vm.provision "ansible" do |ansible|
          # Disable default limit to connect to all the machines
          ansible.lim

* `autostart: false` - Tells Vagrant to not start specific machines.
    * In example above, we start DC1 clusters automatically with `up`.
    * Clients and DC2 we can start explicitly.

Options
* `config.vm.network "public_network", bridge: "eno1"`

## Run vagrant up with Ansible playbook

Start the environment. `--provision` tells Vagrant to provision even if the VM's are already up.

In [156]:
# vagrant halt
pushd tmp
time vagrant up --provision > /tmp/vagrant_up.out 2>&1 &
popd

/media/code/hc_demos-jupyter/tmp /media/code/hc_demos-jupyter
[1] 78921
/media/code/hc_demos-jupyter


Check on progress of `vagrant up`. Takes about 5 minutes.

In [176]:
tail /tmp/vagrant_up.out

changed: [dc1-s-1]

TASK [base : Install base apps] ************************************************
changed: [dc1-s-1] => (item=jq)
ok: [dc1-s-1] => (item=git)
changed: [dc1-s-1] => (item=tree)
changed: [dc1-s-1] => (item=unzip)
ok: [dc1-s-1] => (item=wget)
ok: [dc1-s-1] => (item=curl)
changed: [dc1-s-1] => (item=net-tools)


You can tail the log file from a different terminal.
```shell
tail -f /tmp/vagrant_up.out
```

In [ ]:
vagrant status
pwd

### Verify that the VM's from DC1 are up

In [40]:
pushd tmp
for i in {1..1}; do
vagrant ssh dc1-s-${i} -- bash <<"EOF"
printf "\033[7;31m$(hostname)\033[0m\n"
printf "\033[7;32m#==> Show interfaces\033[0m\n"
ip a | grep -E "UP|inet " | awk '{print $1 $2}'
printf "\033[7;32m#==> Show Hashi Versions\033[0m\n"
consul version | head -n 1; nomad version; vault version
echo
EOF
done
popd

/media/code/hc_demos-jupyter/tmp /media/code/hc_demos-jupyter
dc1-s-1
#==> Show interfaces
1:lo:
inet127.0.0.1/8
2:enp0s3:
inet10.0.2.15/24
3:enp0s8:
inet192.168.17.111/24
#==> Show Hashi Versions
bash: line 5: consul: command not found
bash: line 5: nomad: command not found
bash: line 5: vault: command not found

/media/code/hc_demos-jupyter


In [ ]:
for i in {1..3}; do
vagrant ssh dc2-s-${i} -- bash <<EOF
hostname
ip a
EOF
done

## Auto-generated Ansible Inventory

Vagrant generates an Ansible inventory file encompassing all of the virtual machines it manages and uses it for provisioning machines.

**Example with the ansible provisioner**

```shell
# Generated by Vagrant

default ansible_host=127.0.0.1 ansible_port=2200 ansible_user='vagrant' ansible_ssh_private_key_file='/home/.../.vagrant/machines/default/virtualbox/private_key'
```

In [ ]:
cat .vagrant/provisioners/ansible/inventory/vagrant_ansible_inventory

NOTE: The generated inventory file is stored as part of your local Vagrant environment in `.vagrant/provisioners/ansible/inventory/vagrant_ansible_inventory`.

## Run Ansible manually

For testing, you could run Ansible separately from Vagrant. We will point to the inventory file created by Vagrant.

Create Ansible configuration file that tells ansible not to SSH host key checking.

In [ ]:
cat > ansible.cfg <<EOF
[defaults]
host_key_checking = no
EOF

### Perform a dry run of Ansible.

In [ ]:
ansible-playbook -C \
  -i .vagrant/provisioners/ansible/inventory/vagrant_ansible_inventory \
  ../../../hashistack-ansible/ansible/play_jupyter.yml

### Perform an actual run of Ansible.

In [42]:
ls -al tmp/

total 364
drwx------ 1 pephan pephan   4096 May 27 11:46 .
drwxr-xr-x 1 pephan pephan   4096 May 27  2022 ..
-rwx------ 1 pephan pephan    328 Jul  8  2021 admin-1.token
drwx------ 1 pephan pephan   4096 Dec 22 14:12 config
drwx------ 1 pephan pephan   4096 Dec 22 14:13 is-terraform-aws-tfe-standalone-20210603
-rwx------ 1 pephan pephan 336218 Jun  3  2021 is-terraform-aws-tfe-standalone-20210603.zip
-rwx------ 1 pephan pephan      0 Jul  8  2021 renewable-admin.token
drwxrwxr-x 1 pephan pephan   4096 May 27 12:20 .vagrant
-rw-rw-r-- 1 pephan pephan   1166 May 27 12:20 Vagrantfile
-rw-rw-r-- 1 pephan pephan   1128 May 27 11:36 vars_setup_ubuntu.yml


In [110]:
pwd
ls -al tmp/.vagrant/machines
cat tmp/.vagrant/provisioners/ansible/inventory/vagrant_ansible_inventory

/media/code/hc_demos-jupyter
total 12
drwxrwxr-x 1 pephan pephan 4096 May 27 12:20 .
drwxrwxr-x 1 pephan pephan 4096 May 27 14:15 ..
drwxrwxr-x 1 pephan pephan 4096 May 27 12:20 dc1-s-1
# Generated by Vagrant

dc1-s-1 ansible_host=127.0.0.1 ansible_port=2222 ansible_user='vagrant' ansible_ssh_private_key_file='/media/code/hc_demos-jupyter/tmp/.vagrant/machines/dc1-s-1/virtualbox/private_key'

[dc1_clients]
dc1-c-[1:2]

[all:vars]
arch=amd64


In [70]:
cd ../hashistack-ansible/ansible/

In [69]:
cd -

/media/code/hc_demos-jupyter


In [142]:
pushd ../hashistack-ansible/ansible
ansible-playbook -v \
  -i /media/code/hc_demos-jupyter/tmp/.vagrant/provisioners/ansible/inventory/vagrant_ansible_inventory \
  play_install_jupyter.yml
popd
# ansible-playbook -v \
#   -i /media/code/hc_demos-jupyter/tmp/.vagrant/provisioners/ansible/inventory/vagrant_ansible_inventory \
#   /media/code/hashistack-ansible/ansible/play_install_jupyter.yml

/media/code/hashistack-ansible/ansible /media/code/hc_demos-jupyter
Using /media/code/hashistack-ansible/ansible/ansible.cfg as config file

PLAY [dc1-s-1] *****************************************************************

TASK [Gathering Facts] *********************************************************
ok: [dc1-s-1]

TASK [Start JupyterLab] ********************************************************
changed: [dc1-s-1] => {"changed": true, "cmd": "export PATH=\"$HOME/.local/bin:$PATH\"\nnohup /home/vagrant/.local/bin/jupyter lab --ip=0.0.0.0 --port=8888 --no-browser --notebook-dir=/home/vagrant >> /tmp/jupyterlab.out 2>&1 &\n", "delta": "0:00:00.048263", "end": "2022-05-28 01:30:42.830972", "msg": "", "rc": 0, "start": "2022-05-28 01:30:42.782709", "stderr": "", "stderr_lines": [], "stdout": "", "stdout_lines": []}

PLAY RECAP *********************************************************************
dc1-s-1                    : ok=2    changed=1    unreachable=0    failed=0    skipped=0    res

View ansible logs.

In [ ]:
tail -f /tmp/ansible.log

In [151]:
pushd tmp
vagrant ssh dc1-s-1 -- sh <<EOF
#tail /tmp/ansible.out
printf "\n\n#==> Grab the token from the following output.\n"
tail /tmp/jupyterlab.out | grep -A2 URLs
EOF
popd

/media/code/hc_demos-jupyter/tmp /media/code/hc_demos-jupyter


#==> Grab the token from the following output.
    Or copy and paste one of these URLs:
        http://dc1-s-1:8888/lab?token=25a7e944d55cc2ee978a6d481e1c697ddc452b7d1fb49c3c
     or http://127.0.0.1:8888/lab?token=25a7e944d55cc2ee978a6d481e1c697ddc452b7d1fb49c3c
/media/code/hc_demos-jupyter


### Connect to JupyterLab

Connect to http://localhost:8888/lab?token=<your_token\>

# Clean Up

Destroy all the Vagrant VM's without confirming.

In [152]:
pushd tmp
vagrant destroy -f
popd

/media/code/hc_demos-jupyter/tmp /media/code/hc_demos-jupyter
==> dc1-s-1: Forcing shutdown of VM...
==> dc1-s-1: Destroying VM and associated drives...
/media/code/hc_demos-jupyter


optional - Uninstall VirtualBox.

In [ ]:
sudo apt -y remove virtualbox virtualbox-*

# References

* Ansible - https://www.vagrantup.com/docs/provisioning/ansible
    * Ansible Local Provisioner https://www.vagrantup.com/docs/provisioning/ansible_local
* [Building a HashiCorp Nomad Cluster Lab using Vagrant and VirtualBox](https://discoposse.com/2019/11/21/building-a-hashicorp-nomad-cluster-lab-using-vagrant-and-virtualbox/)
* Nomad Vagrant Lab - https://github.com/discoposse/nomad-vagrant-lab
* [DO Community - Ansible Playbooks](https://github.com/do-community/ansible-playbooks)


## Time

| task | time | comments |
| --- | --- | --- |
| install virtual box | minutes | on ubuntu vm
| vagrant up | 5 minutes | from scratch